In [188]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import accuracy_score,r2_score,confusion_matrix,classification_report,recall_score,confusion_matrix,mean_squared_error
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split,KFold,cross_val_score,ShuffleSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
import re
from pandas_profiling import ProfileReport
pd.set_option("display.precision", 6)
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


In [189]:
df = pd.read_csv('Jan_2020_ontime.csv')
df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
0,1,3,EV,20366,EV,N48901,4397,13930,1393007,ORD,...,GRB,1003.0,0.0,1000-1059,1117.0,0.0,0.0,0.0,174.0,NaN
1,1,3,EV,20366,EV,N16976,4401,15370,1537002,TUL,...,ORD,1027.0,0.0,1000-1059,1216.0,0.0,0.0,0.0,585.0,NaN
2,1,3,EV,20366,EV,N12167,4404,11618,1161802,EWR,...,TYS,1848.0,0.0,1800-1859,2120.0,0.0,0.0,0.0,631.0,NaN
3,1,3,EV,20366,EV,N14902,4405,10781,1078105,BTR,...,IAH,1846.0,0.0,1800-1859,2004.0,0.0,0.0,0.0,253.0,NaN
4,1,3,EV,20366,EV,N606UX,4407,14524,1452401,RIC,...,IAH,1038.0,0.0,1000-1059,1330.0,0.0,0.0,0.0,1157.0,NaN


In [190]:
df.describe()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_TIME,DEP_DEL15,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
count,607346.000000,607346.000000,607346.000000,607346.000000,607346.000000,6.073460e+05,607346.000000,6.073460e+05,600682.000000,600647.000000,600271.000000,599268.000000,607346.000000,607346.000000,607346.000000,0.0
mean,16.014354,3.955735,19994.137492,2622.365261,12657.389167,1.265743e+06,12657.196320,1.265724e+06,1331.512559,0.136326,1477.968924,0.137309,0.011407,0.001893,798.022341,NaN
std,8.990719,1.910205,375.754232,1822.545302,1524.407203,1.524405e+05,1524.279269,1.524277e+05,497.095168,0.343135,525.543294,0.344174,0.106193,0.043473,587.282639,NaN
min,1.000000,1.000000,19393.000000,1.000000,10135.000000,1.013506e+06,10135.000000,1.013506e+06,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,31.000000,NaN
25%,8.000000,2.000000,19790.000000,1070.000000,11292.000000,1.129202e+06,11292.000000,1.129202e+06,919.000000,0.000000,1058.000000,0.000000,0.000000,0.000000,369.000000,NaN
50%,16.000000,4.000000,19977.000000,2177.000000,12889.000000,1.288903e+06,12889.000000,1.288903e+06,1326.000000,0.000000,1513.000000,0.000000,0.000000,0.000000,641.000000,NaN
75%,24.000000,5.000000,20378.000000,4108.000000,14027.000000,1.402702e+06,14027.000000,1.402702e+06,1739.000000,0.000000,1918.000000,0.000000,0.000000,0.000000,1037.000000,NaN
max,31.000000,7.000000,20452.000000,6860.000000,16869.000000,1.686901e+06,16869.000000,1.686901e+06,2400.000000,1.000000,2400.000000,1.000000,1.000000,1.000000,5095.000000,NaN


In [191]:
# comnbining the arrival and departure delay in one column 
df['DELAYED'] = (df['ARR_DEL15'].astype(bool) | df['DEP_DEL15'].astype(bool)).astype(int)

In [192]:
# dropping unuseful columns
df.drop(['OP_CARRIER_AIRLINE_ID','TAIL_NUM','OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID','DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID','Unnamed: 21','OP_CARRIER','ARR_DEL15','DEP_DEL15','CANCELLED', 'DIVERTED'], axis=1, inplace=True)

In [193]:
# getting the object columns
strings_columns =  list(df.dtypes[df.dtypes == 'object'].index)
print(f"The number of string columns is: {len(strings_columns)}")
    
# getting int or float columns
numeric_columns = list(df.drop(strings_columns,axis=1))
print(f"The number of numeric columns is: {len(numeric_columns)}")


The number of string columns is: 4
The number of numeric columns is: 6


In [194]:
df.isna().sum() 

DAY_OF_MONTH            0
DAY_OF_WEEK             0
OP_UNIQUE_CARRIER       0
ORIGIN                  0
DEST                    0
DEP_TIME             6664
DEP_TIME_BLK            0
ARR_TIME             7075
DISTANCE                0
DELAYED                 0
dtype: int64

In [195]:
df.dropna(inplace=True)
df.isna().any()

DAY_OF_MONTH         False
DAY_OF_WEEK          False
OP_UNIQUE_CARRIER    False
ORIGIN               False
DEST                 False
DEP_TIME             False
DEP_TIME_BLK         False
ARR_TIME             False
DISTANCE             False
DELAYED              False
dtype: bool

In [196]:
df.shape

(600271, 10)

In [197]:
df.duplicated().sum()

1

In [198]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

## function to check outliers with box plot

In [199]:
def check_outliers(df):
    for col in numeric_columns:
        fig,ax= plt.subplots(figsize=(10,5))
        plt.grid()
        sns.boxplot(x = df[col])
        plt.show()


## Function to replace outliers with mean

In [200]:
def replace_outliers(df):
    for column in numeric_columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3-q1
        upper_lim = q3 + 1.5 * iqr
        lower_lim = q1 - 1.5 * iqr
        column_mean = df[column].mean()
        outliers_down = (df[column] < lower_lim)
        outliers_up = (df[column] > upper_lim)
        df[column] = np.where((df[column] > upper_lim) | (df[column] < lower_lim) , column_mean, df[column])
        df[column] = pd.DataFrame(df[column],columns=[column])
    return (df)
#replace_outliers(df)

In [201]:
def encode_categories(features):
    lb_make = LabelEncoder()
    for i in range(len(features)):
        df[features[i]] = lb_make.fit_transform(df[features[i]])

## after preparing data, we have to make the final check before entering modeling phase

In [202]:
df.isna().sum()

DAY_OF_MONTH         0
DAY_OF_WEEK          0
OP_UNIQUE_CARRIER    0
ORIGIN               0
DEST                 0
DEP_TIME             0
DEP_TIME_BLK         0
ARR_TIME             0
DISTANCE             0
DELAYED              0
dtype: int64

In [203]:
df.duplicated().sum()

0

In [204]:
check_outliers(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matp

In [205]:
encode_categories(['OP_UNIQUE_CARRIER' , 'ORIGIN' , 'DEST' , 'DEP_TIME_BLK'])

In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600270 entries, 0 to 607345
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DAY_OF_MONTH       600270 non-null  int64  
 1   DAY_OF_WEEK        600270 non-null  int64  
 2   OP_UNIQUE_CARRIER  600270 non-null  int64  
 3   ORIGIN             600270 non-null  int64  
 4   DEST               600270 non-null  int64  
 5   DEP_TIME           600270 non-null  float64
 6   DEP_TIME_BLK       600270 non-null  int64  
 7   ARR_TIME           600270 non-null  float64
 8   DISTANCE           600270 non-null  float64
 9   DELAYED            600270 non-null  int64  
dtypes: float64(3), int64(7)
memory usage: 50.4 MB


## Modeling

In [207]:
X = df.drop('DELAYED',axis=1)
y = df['DELAYED']

In [208]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)

In [209]:
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('RandomForest', RandomForestClassifier(n_estimators=50)))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))


In [210]:
for name, model in models:
    
    print(name)
    
    # Make predictions.
    
    trained_model = model.fit(x_train, y_train)
        
    predictions = trained_model.predict(x_test) 
    
    print(f"train score: {accuracy_score(y_train, trained_model.predict(x_train))}\n")
    
    print(f"test score: {accuracy_score(predictions,y_test)}\n\n")
    

LogisticRegression


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


train score: 0.8269897593701881

test score: 0.830159761440685


RandomForest
train score: 0.9997072745835802

test score: 0.9150049144551619


Decision Tree
train score: 1.0

test score: 0.8835024239092408


KNN
train score: 0.8951400441230017

test score: 0.8606571487275171




## The Random Forest Classifier is chosen for our model

In [211]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

clf = RandomForestClassifier(n_estimators=50, random_state=47).fit(x_train, y_train)

# sum(y_test)
# clf.score(X_test, y_test)

y_predicted = clf.predict(x_test)
confusion = confusion_matrix(y_test, y_predicted)
#confusion
#sum(y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)
print('Feature importances: {}'.format(clf.feature_importances_))

Recall: 0.545
Precision: 0.925
Accuracy: 0.915
F1: 0.686
[[148154   1343]
 [ 13901  16683]]
Feature importances: [0.04882227 0.02932562 0.04748965 0.0819847  0.08195788 0.28740713
 0.14625871 0.15750187 0.11925217]
